In [1]:
from brian2 import *
import numpy as np

start_scope()

defaultclock.dt = 0.0001*ms  

# Custom timing function
@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, result=1, sum=1, spikes_received=1)
def spike_timing(w, global_clock, layer, sum, spikes_received): 
    #print(global_clock)
    x = global_clock % 1
    if w >= 0:
        return (x ** (1 - w)) 
    else:
        return (1 - (1 - x) ** (1 + w)) 
    
@implementation('numpy', discard_units=True)
@check_units(layer=1, result=1, sum=1, spikes_received=1)
def math1(layer, sum, spikes_received): 
    return (sum/spikes_received )+ layer


In [55]:
def create_3layer_SNN(n_input, n_hidden, n_output):
    n_total = n_input + n_hidden + n_output

    neurons = NeuronGroup(n_total, '''
        v : 1
        sum : 1
        spikes_received : 1
        scheduled_time : second
        global_clock : 1
    ''', threshold='v > 1', reset='v = 0', method='exact')

    neurons.v = 0
    neurons.scheduled_time = 1e9 * second
    neurons.global_clock = 0.0
    neurons.sum = 0.0
    neurons.spikes_received = 0.0

    # Dummy stim for wiring
    dummy_stim = SpikeGeneratorGroup(n_input, [], []*ms)

    syn_input = Synapses(dummy_stim, neurons[0:n_input], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
    ''')
    syn_input.connect(j='i')
    syn_input.w = 0.5
    syn_input.layer = 0

    syn_hidden = Synapses(neurons[0:n_input], neurons[n_input:n_input+n_hidden], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
    ''')
    syn_hidden.connect()
    syn_hidden.w = 0.5
    syn_hidden.layer = 1

    syn_output = Synapses(neurons[n_input:n_input+n_hidden], neurons[n_input+n_hidden:n_total], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock, layer, spikes_received, sum)
        scheduled_time = ((sum/spikes_received) + layer) * ms 
    ''')
    syn_output.connect()
    syn_output.w = 0.5
    syn_output.layer = 2

    neurons.run_regularly('''
        v = int(abs(t - scheduled_time) < 0.0005*ms) * 1.2
        global_clock += 0.001
    ''', dt=0.001*ms)

    # Return all components in a dict
    return {
        'neurons': neurons,
        'syn_input': syn_input,
        'n_total': n_total,
        'n_input': n_input
    }


def run_SNN(inputs, snn):
    n_input = snn['n_input']
    n_total = snn['n_total']
    neurons = snn['neurons']
    syn_input = snn['syn_input']

    # Create new SpikeGeneratorGroup for this run
    indices = list(range(n_input))
    stim = SpikeGeneratorGroup(n_input, indices=indices, times=inputs * ms)

    # Rewire synapse to new stim
    syn_input.source = stim

    # Reset state
    neurons.v = 0
    neurons.sum = 0
    neurons.spikes_received = 0
    neurons.scheduled_time = 1e9 * second
    neurons.global_clock = 0.0

    # Add everything into a fresh Network
    net = Network()
    net.add(stim, neurons, syn_input)
    net.add(snn['syn_input'], snn['neurons'])
    spikemon = SpikeMonitor(neurons)
    net.add(spikemon)

    # Run and return results
    net.run(5*ms)

    print("--- New Input Run Results ---")
    for i in range(n_total):
        times = spikemon.spike_trains()[i]
        formatted = [f"{t/ms:.3f} ms" for t in times]
        print(f"Neuron {i} spike times: {formatted}")



In [56]:
snn = create_3layer_SNN(2, 2, 2)

run_SNN(np.array([0.1, 0.5]), snn)
run_SNN(np.array([0.2, 0.4]), snn)


WARNING    'i' is an internal variable of group 'synapses_6', but also exists in the run namespace with the value 4. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    The object 'synapses_7' is getting deleted, but was never included in a network. This probably means that you did not store the object reference in a variable, or that the variable was not used to construct the network.
The object was created here (most recent call only):
  File 'C:\Users\irtho\AppData\Local\Temp\ipykernel_16972\3363048880.py', line 33, in create_3layer_SNN
    syn_hidden = Synapses(neurons[0:n_input], neurons[n_input:n_input+n_hidden], ''' [brian2.core.base.unused_brian_object]
WARNING    The object 'synapses_1' is getting deleted, but was never included in a network. This probably means that you did not store the object reference in a variable, or that the variable was not used to construct the network.
The object was created here (most recent call on

ValueError: 'synapses_6' has been included in the network but not the object on which it depends.

In [18]:
print([range(0,5)])
indices = []



[range(0, 5)]
